In [ ]:
from ase import Atoms
import ase.units as units
from ase.io import read
from ase.build import bulk
from ase.calculators.espresso import Espresso

import numpy as np
import matplotlib.pyplot as plt

import os
import dotenv

dotenv.load_dotenv()
PWSCF_COMMAND = os.environ.get("PWSCF_COMMAND")
PPDIR = os.environ.get("PSEUDOPOTENTIALS")

outdir = "./results"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
k_points = 3

# Setup DFT calculation parameters
pseudopotentials = {'Fe': 'Fe.pbe-nd-rrkjus.UPF'}  # Example, change as needed

calc_params = {
    'calculation': 'vc-relax',
    'tprnfor': True,
    'tstress': True,
    'input_data': {
        'system': {
            'smearing': 'mp',
            'occupations': 'smearing',
            'starting_magnetization': 1,
            'degauss': 0.02,
            'nspin': 2,
            'ntyp': 1,
            'nat': 8,
            'ecutwfc': 30,  # PW cutoff
            'ecutrho': 300,  # Charge cutoff
            'ibrav': 1,
        },
        'electrons': {
            'diagonalization': 'david',
            'mixing_beta': 0.5,
            'conv_thr': 1e-07,
        },
#         'cell': {
#             'press': 250,
#             'press_conv_thr': 0.01,
#             'cell_dynamics': 'bfgs',
#             'cell_dofree':   'all',
#         }
    },
    'pseudopotentials': pseudopotentials,
    'kpts': (k_points, k_points, k_points),  # k-points grid, adjust as needed
    'parallel': 'all',
    'directory': outdir,  # Custom directory for calculation files
    'label': "Fe",  # Prefix for the filenames
    'logfile': "Fe.log",  # Logfile name
    'command': "mpirun -np 16 " + PWSCF_COMMAND + " -in Fe.pwi > Fe.pwo",
    'pseudo_dir': PPDIR,
}

In [ ]:
a = 2.45

energies_bcc = []
lattice_params = []

iron_bcc = Atoms('Fe2', 
              scaled_positions=[(0, 0, 0), 
                                (0.5, 0.5, 0.5)], 
              cell=[a, a, a])
    
# Attach the calculator to the structures
calculator = Espresso(**calc_params)
iron_bcc.set_calculator(calculator)
energy_bcc = iron_bcc.get_potential_energy()
energies_bcc.append(energy_bcc)

final_structure = read('results/Fe.pwo') 

# Extracting the lattice constants
lattice_constants = final_structure.cell.lengths()
lattice_params.append(lattice_constants)

# Conditions for antiferromagnetism
cell_params_anti = {
    'press': 110,
    'press_conv_thr': 0.5,
    'cell_dynamics': 'bfgs',
    'cell_dofree':   'all',
}
calc_params['input_data']['cell'] = cell_params_anti
calc_params['label'] = "Fe2"
calc_params['logfile'] = "Fe2.log"
calc_params['command'] = "mpirun -np 16 " + PWSCF_COMMAND + " -in Fe2.pwi > Fe2.pwo",
iron_bcc = Atoms('Fe2', 
              scaled_positions=[(0, 0, 0), 
                                (0.5, 0.5, 0.5)], 
              cell=[a, a, a])

print (calc_params)

calculator = Espresso(**calc_params)
iron_bcc.set_calculator(calculator)
energy_bcc = iron_bcc.get_potential_energy()
energies_bcc.append(iron_bcc.get_potential_energy())

final_structure = read('results/Fe2.pwo') 

# Extracting the lattice constants
lattice_constants = final_structure.cell.lengths()
lattice_params.append(lattice_constants)


cell_params_non = {
    'press': 250,
    'press_conv_thr': 0.5,
    'cell_dynamics': 'bfgs',
    'cell_dofree':   'all',
}
calc_params['input_data']['cell'] = cell_params_non
calc_params['label'] = "Fe3"
calc_params['logfile'] = "Fe3.log"
calc_params['command'] = "mpirun -np 16 " + PWSCF_COMMAND + " -in Fe3.pwi > Fe3.pwo",
iron_bcc = Atoms('Fe2', 
              scaled_positions=[(0, 0, 0), 
                                (0.5, 0.5, 0.5)], 
              cell=[a, a, a])

calculator = Espresso(**calc_params)
iron_bcc.set_calculator(calculator)
energy_bcc = iron_bcc.get_potential_energy()
energies_bcc.append(iron_bcc.get_potential_energy())

final_structure = read('results/Fe3.pwo') 

# Extracting the lattice constants
lattice_constants = final_structure.cell.lengths()
lattice_params.append(lattice_constants)


In [ ]:
# Average lattice constants for each pressure
avg_lattice_constants = [sum(lc) / len(lc) for lc in lattice_params]

# Creating the plot
fig, ax1 = plt.subplots()

pressures = [0, 110, 250]

# Plotting energy
color = 'tab:red'
ax1.set_xlabel('Pressure (kbar)')
ax1.set_ylabel('Energy (meV)', color=color)
ax1.plot(pressures, energies_bcc, color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Creating a second y-axis for lattice constant
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Lattice Constant (Angstrom)', color=color)
ax2.plot(pressures, avg_lattice_constants, color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Title and layout
plt.title('Energy and Lattice Constant vs Pressure')
fig.tight_layout()

# Show plot
plt.show()